In [7]:
from facenet_pytorch import InceptionResnetV1, MTCNN
# from mtcnn import MTCNN
from mtcnn.detector import MtcnnDetector
import cv2
from align_faces import warp_and_crop_face, get_reference_facial_points
import numpy as np
from numpy.linalg import norm
import time
import glob
import torch 
from torchvision import transforms
from facenet_pytorch import InceptionResnetV1, fixed_image_standardization
import os
from PIL import Image
import numpy as np

In [16]:
mtcnn = MTCNN()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = InceptionResnetV1(
    classify=False,
    pretrained="casia-webface"
).to(device)

infor = model.eval()

In [1]:
from scipy import spatial

dataSetI = [3, 45, 7, 2]
dataSetII = [2, 54, 13, 15]
result = 1 - spatial.distance.cosine(dataSetI, dataSetII)

In [2]:
print(result)

0.97228425171235


In [ ]:
from re import X
import threading
import cv2
from facenet_pytorch import InceptionResnetV1
# import tensorflow as tf
import time 
import torch
# import tensorflow as tf
# from tensorflow.keras.models import load_model
import pickle
import cv2
import numpy as np
import cv2
from torchvision import transforms
from align_faces import warp_and_crop_face, get_reference_facial_points
from mtcnn.detector import MtcnnDetector
detector = MtcnnDetector()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

def trans(img):
    transform = transforms.Compose([
            transforms.ToTensor(),
            fixed_image_standardization
        ])
    return transform(img).unsqueeze(0)
def fixed_image_standardization(image_tensor):
    processed_tensor = (image_tensor - 127.5) / 128.0
    return processed_tensor

model_resnet = InceptionResnetV1(
    classify=False,
    pretrained="casia-webface"
).to(device)
model_resnet.eval()

def embed_face(img):
    img= cv2.resize(img,(160,160))
    with torch.no_grad():
        embed = model_resnet(trans(img).to(device))
    return embed

model = pickle.load(open("Model/2022_10_02_19-32-49_SVM.pkl","rb"))
names = pickle.load(open("List_user/2022_10_01_11-32-55_ListUser.pkl","rb"))

probability = 0.87

def indentify(lst_face):
    x = time.time()
    out = 1-model.predict_proba(np.asarray(lst_face))
    lst_out = []
    lst_prob = []
    for x in out:
        if(np.max(x)>=probability):
            lst_out.append(np.argmax(out))
            lst_prob.append(np.max(x))
        else:
            lst_out.append(-1)
            lst_prob.append(np.max(x))
    print(lst_prob)
    print(lst_out)
    print(out)
    
    return lst_out,lst_prob


def predict_face(embed_img):
    out = 1 - model.predict_proba(embed_img.reshape(1,-1))
    proba = np.max(out)
    print(out)
    id = np.argmax(out)
    if(proba<probability):
        return "unknow", proba
    else:
        return names[id], proba




def mask_detect(image):
    # img = image.copy()
    img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    print(img.shape)
    (h,w) = img.shape[:2]
    boxes, facial5points = detector.detect_faces(img)
    lst_embeddings = []
    lst_max = []
    lst_out = []
    if(len(boxes)!=0):

        for box in boxes:
            (startX,startY,endX,endY)=box[:4].astype('int')
            #ensure the bounding boxes fall within the dimensions of the frame
            (startX,startY)=(max(0,startX),max(0,startY))
            (endX,endY)=(min(w-1,endX), min(h-1,endY))

            #extract the face ROI, convert it from BGR to RGB channel, resize it to 224,224 and preprocess it
            face=img[startY:endY, startX:endX]
            embed = embed_face(face).numpy().reshape(512)
            name, proba = predict_face(embed)

            if(name == "unknow"):
                color = (255,0,0)

                label="{}: {:.2f}%".format(name,proba*100)
                #display the label and bounding boxes
                cv2.putText(image,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
                cv2.rectangle(image,(startX,startY),(endX,endY),color,2)
            else:
                color = (0,255,0)

                label="{}: {:.2f}%".format(name,proba*100)
                #display the label and bounding boxes
                cv2.putText(image,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
                cv2.rectangle(image,(startX,startY),(endX,endY),color,2)
    return image

global a
a = 10

def PlayCamera(id):    
    video_capture = cv2.VideoCapture(id)
    while True:
        x = time.time()
        ret, frame = video_capture.read()
        # img = frame[0:128,0:128]
        # print(model.predict(np.array([img])))
        img = mask_detect(frame)
        print(time.time() - x)
        cv2.imshow('{}'.format(id), img)        
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    video_capture.release()

cameraIDs = [0]
threads = []
for id in cameraIDs:
    threads += [threading.Thread(target=PlayCamera, args=(id,))]
for t in threads:    
    t.start()
for t in threads: 
    t.join()

In [ ]:
def face_detect(image):
    
    img = image.copy()
    (h,w) = img.shape[:2]
    boxes, facial5points = detector.detect_faces(img)
    for box in boxes:
        (startX,startY,endX,endY)=box[:4].astype('int')

        #ensure the bounding boxes fall within the dimensions of the frame
        (startX,startY)=(max(0,startX),max(0,startY))
        (endX,endY)=(min(w-1,endX), min(h-1,endY))

        #extract the face ROI, convert it from BGR to RGB channel, resize it to 224,224 and preprocess it
        face=img[startY:endY, startX:endX]
        face=cv2.resize(face,(160,160))

        # (mask,withoutMask) = model.predict(face.reshape(1,224,224,3))[0]

        # #determine the class label and color we will use to draw the bounding box and text
        # label='Mask' if mask>withoutMask else 'No Mask'
        # color=(0,255,0) if label=='Mask' else (0,0,255)
        color=(0,255,0)
        #include the probability in the label
        # label="{}: {:.2f}%".format(label,max(mask,withoutMask)*100)
        label = "Truyen"
        #display the label and bounding boxes
        cv2.putText(img,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(img,(startX,startY),(endX,endY),color,2)
        
    return face

In [29]:
out = model(x.unsqueeze(0))
print(out.shape)

torch.Size([1, 512])


In [3]:
# importing the module

import glob
import cv2
 
# reading the video
source = cv2.VideoCapture(0)
 
# running the loop
while True:
 
    # extracting the frames
    ret, img = source.read()
     
    # converting to gray-scale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # displaying the video
    cv2.imshow("Live", gray)
    
    # exiting the loop
    key = cv2.waitKey(1)
    if key == ord("q"):
        break
     
# closing the window
cv2.destroyAllWindows()
source.release()

In [15]:
IMG_PATH = "../Data/Image"
for usr in os.listdir(IMG_PATH):
    embeds = []
    for file in glob.glob(os.path.join(IMG_PATH, usr)+'/*.jpg'):
        try:
            # img = Image.open(file)
            img = cv2.imread(file)
            # img2 = img.copy()
            # print(img.shape)
            
            
            # face = face_detect(img2)
            # gray = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            # x = np.zeros(face.shape)
            # x[:,:,0] = gray
            # x[:,:,1] = gray
            # x[:,:,2] = gray

            cv2.imshow("face", img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        except:
            continue